In [4]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Data wash


In [ ]:
path = '../data/playground-series-s4e10/train.csv'

df = pl.read_csv(path)
df.head()
new_values = {
    'DEBTCONSOLIDATION': 4,
    'PERSONAL': 2,
    'VENTURE': 0,
    'HOMEIMPROVEMENT': 3,
    'MEDICAL': 3,
    'EDUCATION': 1,
    'N': 0,
    'Y': 1
}


df = df.with_columns(pl.col('cb_person_default_on_file').replace(new_values).alias('cb_person_default_on_file'))
column_unique = sorted(df['loan_grade'].unique())[::-1]
test = {column_unique[i]: int(i) for i in range(len(column_unique))}
df = df.with_columns(pl.col('loan_grade').replace(test).cast(pl.Int8, strict=True).alias('loan_grade'))
df = df.with_columns(pl.col('cb_person_default_on_file').cast(pl.Int8, strict=True).alias('cb_person_default_on_file'))
df = df.filter(pl.col('person_income') <= 2.5*1e5)
df = df.with_columns(pl.col('loan_intent').replace(new_values).alias('loan_intent'))
df = df.with_columns(pl.col('loan_intent').cast(pl.Int8, strict=True).alias('loan_intent'))
df.head()

In [ ]:
print(df['person_home_ownership'].unique())
values = df['person_home_ownership'].unique()

for val in values:
    print(len(df.filter(pl.col(df['person_home_ownership'] == val)).filter(pl.col(df['loan_status'] == 1)))/len(df.filter(pl.col(df['person_home_ownership'] == val))), val)

In [ ]:
# Get unique values from the 'person_home_ownership' column
values = df['person_home_ownership'].unique()

# Initialize an empty dictionary to store the results
ratios_dict = {}

# Calculate and store the ratios in the dictionary
for val in values:
    count_loan_status = len(df.filter(pl.col('person_home_ownership') == val).filter(pl.col('loan_status') == 1))
    total_count = len(df.filter(pl.col('person_home_ownership') == val))

    # Avoid division by zero
    if total_count > 0:
        ratio = count_loan_status / total_count
        ratios_dict[val] = ratio
    else:
        ratios_dict[val] = None  # or you can choose to omit this entry

# Print the resulting dictionary
print(ratios_dict)


In [ ]:
new_values_person_home_ownership = {'RENT': 1, 'OTHER':ratios_dict["OTHER"]/ratios_dict['RENT'], 'MORTGAGE': ratios_dict['MORTGAGE']/ratios_dict['RENT'], 
                                    'OWN': ratios_dict['OWN']/ratios_dict['RENT']}

# Define new values as strings
new_values_person_home_ownership = {
    'RENT': str(1),  # This will convert the integer 1 to a string "1"
    'OTHER': str(ratios_dict["OTHER"] / ratios_dict['RENT']),
    'MORTGAGE': str(ratios_dict['MORTGAGE'] / ratios_dict['RENT']),
    'OWN': str(ratios_dict['OWN'] / ratios_dict['RENT'])
}

# Display the new values
print(new_values_person_home_ownership)

df = df.with_columns(pl.col('person_home_ownership').replace(new_values_person_home_ownership).cast(pl.Float32))

df

In [ ]:
# corelation = np.ones(len(df.columns) -4)

nr_colums = df.columns
nr_colums.remove('loan_int_rate')
nr_colums.remove('id')
# nr_colums.remove('person_home_ownership')
# nr_colums.remove('loan_intent')
nr_colums.remove('person_age')
nr_colums.remove('loan_status')
nr_colums.remove('cb_person_cred_hist_length')
corelation = np.ones(len(nr_colums))
for ind in range(len(nr_colums)):

    corelation[ind] = df.select(pl.corr('loan_status', nr_colums[ind])).to_numpy()[0][0]


sns.heatmap(corelation.reshape(-1,1), annot = True)


In [ ]:
# Trying to remove some dim from the input 
nr_of_colums = len(nr_colums)

corr_matrix = np.ones((nr_of_colums, nr_of_colums), dtype='float32')

for i in range(nr_of_colums):
    print(f'done with {nr_colums[i]}')
    for j in range(nr_of_colums):
        corr_ft = df.select(pl.corr(nr_colums[i], nr_colums[j]))[0,0]
        corr_matrix[i][j] = corr_ft


print(corr_matrix.shape)

In [143]:
import pandas as pd

In [144]:
corr_input_matrix = pd.DataFrame(corr_matrix, columns=nr_colums, index=nr_colums)




In [ ]:
sns.heatmap(corr_input_matrix, annot= True)
plt.show()

## Tree Claddifier and Log

In [171]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
from torch.optim import Adam



In [168]:
X = df[nr_colums]
Y = df['loan_status']
X = X.to_numpy()
X = torch.tensor(X) 
Y = torch.tensor(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [305]:
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, Y_train)
Y_pred = model_tree.predict(X_test)
print(accuracy_score(Y_pred, Y_test))

0.9128726403006748


## MLP network, 

### the loss function have to be custm because the loss now with just guessing 0 is just to great


### Todo create a loss function punisch the model if geting 1s wrong, (or creat more 1s and remove 0s) and create the training loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [302]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(8, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256,64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))  
        return x  
        

model = MLP()





criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
